In [ ]:
# Andrew Harvey
# A program to create a 3D interactive model of star distribution, displaying all ~9,000 naked-eye visible stars, from the Yale Bright Star Catalouge.
import math
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df = pd.read_csv("bsc5.csv", skiprows = 0)

In [ ]:
n_paralaxCol = 41 # Column number containing n_parallax
parallaxCol = 42 # Column number containing parallax

In [ ]:
# Dropping all objects that do not have a trigonometric parallax
for row in range(len(df.index) - 1, -1, -1):
    n_paralax = df.iloc[row][n_paralaxCol]
    if(n_paralax == "D"):
        df = df.drop(row)
   
df = df.dropna(subset = ["Parallax"])


In [ ]:
# Compute distance
parsecs = []
# Replace zeros in the parallax column to avoid division by zero

for row in range(len(df.index)):
    parallax = df.iloc[row][parallaxCol]
    if(parallax != 0):
        parsecs.append( 1.0 / parallax)
    
df["Distance"] = parsecs


In [ ]:
# Converting RA and DEC from degrees to radians:

ra = [] # array to store right ascension in radians
dec = [] # array to store declination in radians

# 1 hr = 15 degrees
# 1 degree = pi/180

# We can convert RA to degrees by using the formula: degrees = (RA hr) + (RA min)/60 + (RA sec)/3600 * 15
# and then to radians: degrees * pi/180

for row in range(len(df.index)):
    rah = df.iloc[row][19]
    ram = df.iloc[row][20]
    ras = df.iloc[row][21]
    
    RAdegrees = (rah + ram/60 + ras/3600) * 15
    RAradians = RAdegrees * (math.pi / 180)
    
    if RAradians > math.pi:
        ra.append(RAradians - (2 * math.pi))
    else:
        ra.append(RAradians)
    
    
for row in range(len(df.index)):
    sign = df.iloc[row][22]
    ded = df.iloc[row][23]
    dem = df.iloc[row][24]
    des = df.iloc[row][25]
    
    DECdegrees = ded + ((dem/60 + des/3600))
    DECradians = DECdegrees * (math.pi / 180)
    
    if(sign == "-"):
        dec.append(DECradians * -1)
    else:
        dec.append(DECradians)

In [ ]:
df["RA radians"] = ra
df["DEC radians"] = dec

In [ ]:
# This cell performs coordinate transformation from a spherical coordinate system to a cartesian system.
# x-axis: toward vernal equinox
# x-y plane: plane of the celestial equator
# z-axis: toward the celestial equator 

# Columns x, y, and z will have the distance to the stars in parsecs in the equatorial frame of reference.
x = []
y = []
z = []
distanceCol = 53
raCol = 54
decCol = 55
# Compute 3D Coordinates
for row in range(len(df.index)):
    distance = df.iloc[row][distanceCol]
    ra = df.iloc[row][raCol]
    dec = df.iloc[row][decCol]
    
    x.append(distance * np.cos(dec) * np.cos(ra))
    y.append(distance * np.cos(dec) * np.sin(ra))
    z.append(distance * np.sin(dec))
    
# Add x, y, z columns to the dataframe
df["x"] = x
df["y"] = y
df["z"] = z

In [ ]:
# Get the colors for each star
# The stars on the diagram will be color-coded based on their spectral classification using the Morgan–Keenan (MK) system.
colors = []
spectral_type_col = 37
for row in range(len(df.index)):
    spectral_type = df.iloc[row][spectral_type_col]
    star_color = spectral_type[0:1]
    if(star_color == "O"):
        colors.append("midnightblue")
    elif(star_color == "B"):
        colors.append("mediumblue")
    elif(star_color == "A"):
        colors.append("lightblue")
    elif(star_color == "F"):
        colors.append("green")
    elif(star_color == "G"):
        colors.append("yellow")
    elif(star_color == "K"):
        colors.append("orange")
    elif(star_color == "M"):
        colors.append("red")
    else:
        colors.append("black")
        
df["color"] = colors 
    

In [ ]:
# Creates a 3D distribution of the stars based on their distance from earth.
# Hovering over a star in the model will display its position, HR number, distance (PC), and radial velocity (km/s).
fig = px.scatter_3d(data_frame = df, x = df["x"], y = df ["y"], z = df["z"], 
                    hover_data = ["HR", "Distance", "RadVel"], 
                    color = df["color"], color_discrete_map="identity")

fig.update_layout(scene = dict(
                    xaxis_title='X (pc), + towards vernal equinox',
                    yaxis_title='Y (pc)',
                    zaxis_title='Z (pc), + towards NCP'),
                    width=900,
                    height=900
                  )

fig.update_traces(marker=dict(size=2))
# Enjoy the view of our local universe!
fig.show()